In [1]:
import pandas as pd

circuit_geometry = pd.read_csv('csv_output/Circuit_geometry_unique.csv')

print("Circuit Geometry Dataset")
print("="*70)
print(f"Shape: {circuit_geometry.shape}")
print(f"Columns: {circuit_geometry.columns.tolist()}")
print(f"\nUnique circuits: {circuit_geometry['circuitRef'].nunique()}")



########################################################
# LOAD COMBINED LAP TIMING CLEAN
########################################################

lap_timing = pd.read_excel('csv_output/Combined_Lap_Timing_Clean.xlsx')

print("Combined Lap Timing Clean Dataset")
print("="*70)
print(f"Shape: {lap_timing.shape}")
print(f"Columns: {lap_timing.columns.tolist()}")
print(f"Total Rows: {len(lap_timing)}")



Circuit Geometry Dataset
Shape: (24, 28)
Columns: ['circuitRef', 'year', 'round', 'name', 'type', 'num_drs_zones', 'length_km', 'num_turns', 'slow_corners', 'medium_corners', 'fast_corners', 'flat_corners', 'slow_cluster_max', 'straight_distance_m', 'straight_ratio', 'straight_len_max_m', 'n_major_straights', 'heavy_braking_zones', 'heavy_braking_mean_dv_kmh', 'hb_spacing_std_m', 'hb_at_end_of_max', 'corner_density_tpkm', 'avg_corner_angle', 'total_corner_angle', 'avg_corner_distance', 'drs_zones_detected', 'drs_total_len_m', 'drs_on_max']

Unique circuits: 24
Combined Lap Timing Clean Dataset
Shape: (67974, 24)
Columns: ['year', 'round', 'Grand_Prix', 'Name', 'RacingNumber', 'Team', 'lap_number', 'LapTime', 'IntervalToPositionAhead', 'Position', 'Stint', 'Compound', 'New', 'AirTemp', 'Humidity', 'Pressure', 'TrackTemp', 'WindDirection', 'WindSpeed', 'Rainfall', 'any_violation', 'lap_clean', 'PitIn', 'PitOut']
Total Rows: 67974


In [5]:
def filter_clean_racing_laps(df):
    """
    Filter the lap timing dataset to keep only clean racing laps.
    
    Removes rows where:
    - Rainfall is True (wet conditions)
    - any_violation is True (track limits, safety car, etc.)
    - lap_clean is False (outlier/invalid laps)
    - PitIn is True (lap entering pit)
    - PitOut is True (lap exiting pit)
    - LapTime is missing/NaN (removes that row AND the next row)
    - Compound is "UNKNOWN"
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The Combined Lap Timing Clean dataset
    
    Returns:
    --------
    pandas.DataFrame
        Filtered dataset with only clean racing laps
    """
    print("Filtering for clean racing laps...")
    print(f"Initial rows: {len(df)}")
    
    # Create a copy to avoid modifying the original
    df_filtered = df.copy()
    
    # Step 1: Identify rows with missing LapTime and mark them + next row for removal
    missing_laptime_mask = df_filtered['LapTime'].isna()
    rows_to_remove = missing_laptime_mask.copy()
    
    # Mark the row AFTER each missing LapTime for removal (x+1)
    missing_indices = df_filtered[missing_laptime_mask].index
    for idx in missing_indices:
        # Find the position of this index in the dataframe
        pos = df_filtered.index.get_loc(idx)
        # If there's a next row, mark it for removal
        if pos + 1 < len(df_filtered):
            next_idx = df_filtered.index[pos + 1]
            rows_to_remove.loc[next_idx] = True
    
    print(f"Rows with missing LapTime: {missing_laptime_mask.sum()}")
    print(f"Total rows to remove (missing + next): {rows_to_remove.sum()}")

    
    # Step 2: Apply all filters - keep rows where all conditions are "clean"
    df_filtered = df_filtered[~rows_to_remove]  # Remove rows with missing LapTime and their next row
    
    initial_after_laptime = len(df_filtered)
    
    df_filtered = df_filtered[
        (df_filtered['Rainfall'] != True) &           # No rain
        (df_filtered['any_violation'] != True) &      # No violations
        (df_filtered['lap_clean'] == True) &          # Lap is clean
        (df_filtered['PitIn'] != True) &              # Not entering pit
        (df_filtered['PitOut'] != True) &             # Not exiting pit
        (df_filtered['Compound'] != 'UNKNOWN')        # No unknown compound
    ]
    
    print(f"Rows removed due to UNKNOWN compound: {initial_after_laptime - len(df_filtered)}")
    print(f"Rows after all filtering: {len(df_filtered)}")
    print(f"Total rows removed: {len(df) - len(df_filtered)} ({100 * (len(df) - len(df_filtered)) / len(df):.2f}%)")
    
    return df_filtered


# Apply the filter to get clean racing laps
lap_timing_clean = filter_clean_racing_laps(lap_timing)

print("\nClean Racing Laps Dataset")
print(f"Shape: {lap_timing_clean.shape}")


lap_timing_clean.to_excel('csv_output/Filtered_Clean_Racing_Laps.xlsx', index=False)

Filtering for clean racing laps...
Initial rows: 67974
Rows with missing LapTime: 2086
Total rows to remove (missing + next): 3589
Rows removed due to UNKNOWN compound: 12088
Rows after all filtering: 52297
Total rows removed: 15677 (23.06%)

Clean Racing Laps Dataset
Shape: (52297, 24)
